In [135]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [136]:
num_wires=4
dev = qml.device("default.qubit", wires=num_wires)

Variational classifiers usually define a “layer” or “block”, which is an elementary circuit architecture that gets repeated to build the full variational circuit.

Our circuit layer will use four qubits, or wires, and consists of an arbitrary rotation on every qubit, as well as a ring of CNOTs that entangles each qubit with its neighbour. Borrowing from machine learning, we call the parameters of the layer weights.

In [137]:
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)   
    
    for wire in ([0,1],[1,2],[2,3],[3,0]):
        qml.CNOT(wire)  #aplyinh CNOT gates to adjacent gates

In [138]:
def state_prep(x):
    qml.BasisState(x, wires=[0,1,2,3])


In [139]:
@qml.qnode(dev)

def circuit(wieghts,x):
    state_prep(x)
    
    for layer_weights in weights:
        layer(layer_weights)
    
    return qml.expval(qml.PauliZ(0))

If we want to add a “classical” bias parameter, the variational quantum classifier also needs some post-processing. We define the full model as a sum of the output of the quantum circuit, plus the trainable bias.

In [140]:
def variational_calssifier(weights, bias, x):
    return circuit(weights,x) + bias

Cost Function

In [141]:
def square_loss(labels, predictions):
    return np.mean((labels-qml.math.stack(predictions))**2)   #loss

def acc(labels, predictions):
    accu = sum(abs(l-p) <2 for l,p in zip(labels, predictions))   #accuracy
    accu =accu/len(labels)
    return accu

In [142]:
def cost(weights, bias, X,Y):
    predictions = [variational_calssifier(weights,bias, x) for x in X]
    return square_loss(Y, predictions)

In [143]:
data = np.array([[0,0,0,1,1],                         #training data
                 [0,0,1,0,1],
                 [0,1,0,0,1],
                 [0,1,0,1,0],
                 [0,1,1,0,0],
                 [0,1,1,1,1],
                 [1,0,0,0,1],
                 [1,0,0,1,0],
                 [1,0,1,1,1],
                 [1,1,1,1,0]])

In [144]:
data.shape

(10, 5)

In [145]:
X = np.array(data[:,:-1])             #seperating labels from other features.
Y = np.array(data[:,-1])

In [146]:
Y = 2*Y -1                             #making labels 0,1 to -1,1

In [147]:
for x,y in zip(X,Y):
    print(f'x:{x}    y:{y}')

x:[0 0 0 1]    y:1
x:[0 0 1 0]    y:1
x:[0 1 0 0]    y:1
x:[0 1 0 1]    y:-1
x:[0 1 1 0]    y:-1
x:[0 1 1 1]    y:1
x:[1 0 0 0]    y:1
x:[1 0 0 1]    y:-1
x:[1 0 1 1]    y:1
x:[1 1 1 1]    y:-1


In [148]:
#np.random.seed(7)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)   #std = 0.01, mean = 0
bias_init = np.array(0.0, requires_grad=True)   #requires_grad=True here means that bias_init trm will be optimized during training

print("Weights:", weights_init)
print("Bias: ", bias_init)

Weights: [[[ 0.00064837 -0.01176363  0.00757532]
  [-0.00217329  0.00377365  0.01429097]
  [ 0.00820614  0.02755201 -0.0051022 ]
  [ 0.00075407 -0.01243131 -0.01678522]]

 [[ 0.00685795 -0.02222703  0.02183569]
  [ 0.00159701  0.0055386  -0.01232673]
  [ 0.01301957  0.00513228  0.00736791]
  [ 0.01150375  0.00057456  0.00676543]]]
Bias:  0.0


In [149]:
opt = NesterovMomentumOptimizer(0.01)            #0.01 is the learning rate
batch_size = 5                                   #taking a batch size of 5

In [150]:
weights = weights_init
bias = bias_init

for i in range(50):
    batch_index = np.random.randint(0, len(X), (batch_size,))  #creates indices and send this indices to next line and
    X_batch = X[batch_index]                #X_batch contains like [X[batch_index1], X[batch_index2],...]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)
    
    predictions = [np.sign(variational_calssifier(weights,bias,x)) for x in X]
    #print(predictions)  using all x in X
    current_cost = cost(weights, bias, X, Y)
    
    accuracy = acc(Y, predictions)
    
    print(f"Iter : {i+1} | Cost : {current_cost:0.4f} | Accuracy : {accuracy:0.3f}")
    
    

Iter : 1 | Cost : 2.0098 | Accuracy : 0.500
Iter : 2 | Cost : 2.0195 | Accuracy : 0.500
Iter : 3 | Cost : 2.0202 | Accuracy : 0.500
Iter : 4 | Cost : 2.0243 | Accuracy : 0.500
Iter : 5 | Cost : 2.0234 | Accuracy : 0.500
Iter : 6 | Cost : 2.0181 | Accuracy : 0.500
Iter : 7 | Cost : 2.0062 | Accuracy : 0.500
Iter : 8 | Cost : 1.9879 | Accuracy : 0.500
Iter : 9 | Cost : 1.9720 | Accuracy : 0.500
Iter : 10 | Cost : 1.9618 | Accuracy : 0.500
Iter : 11 | Cost : 1.9602 | Accuracy : 0.500
Iter : 12 | Cost : 1.9662 | Accuracy : 0.500
Iter : 13 | Cost : 1.9797 | Accuracy : 0.500
Iter : 14 | Cost : 1.9952 | Accuracy : 0.500
Iter : 15 | Cost : 2.0126 | Accuracy : 0.500
Iter : 16 | Cost : 2.0263 | Accuracy : 0.500
Iter : 17 | Cost : 2.0390 | Accuracy : 0.500
Iter : 18 | Cost : 2.0454 | Accuracy : 0.500
Iter : 19 | Cost : 2.0600 | Accuracy : 0.500
Iter : 20 | Cost : 2.0723 | Accuracy : 0.500
Iter : 21 | Cost : 2.0763 | Accuracy : 0.500
Iter : 22 | Cost : 2.0725 | Accuracy : 0.500
Iter : 23 | Cost : 

In [151]:
predictions      #predictions of the last iteration

[tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True)]

In [152]:
test_data = np.array([[0,0,0,0,0],               #test data
                 [0,0,1,1,0],
                 [1,0,1,0,0],
                 [1,1,1,0,1],
                 [1,1,0,0,0],
                 [1,1,0,1,1],
                 [0,1,0,1,1]])

In [153]:
X_test = np.array(test_data[:,:-1])             #seperating the labels
Y_test = np.array(test_data[:,-1])

In [154]:
for x,y in zip(X_test,Y_test):
    print(f'x:{x}    y:{y}')

x:[0 0 0 0]    y:0
x:[0 0 1 1]    y:0
x:[1 0 1 0]    y:0
x:[1 1 1 0]    y:1
x:[1 1 0 0]    y:0
x:[1 1 0 1]    y:1
x:[0 1 0 1]    y:1


In [155]:
Y_test = Y_test*2 - 1

In [157]:
predictions_test = [np.sign(variational_calssifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = acc(Y_test, predictions_test)

print(f"Accuracy on unseen data : {acc_test:0.3f}")

x = [0 0 0 0], y = -1, pred=1.0
x = [0 0 1 1], y = -1, pred=-1.0
x = [1 0 1 0], y = -1, pred=-1.0
x = [1 1 1 0], y = 1, pred=1.0
x = [1 1 0 0], y = -1, pred=1.0
x = [1 1 0 1], y = 1, pred=-1.0
x = [0 1 0 1], y = 1, pred=1.0
Accuracy on unseen data : 0.571
